In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

location = "./clothing_data_processed.csv"
clothing_data = pd.read_csv(location)
clothing_data

In [ ]:
clothing_data = clothing_data.drop(['user_id', 'review_text', 'review_summary'], axis=1)
clothing_data = clothing_data.astype({'bust_size': 'str'})
clothing_data

In [ ]:
import collections

def get_profile(group_by_column, clothing_data_df):
    grouped_df = clothing_data_df.groupby([group_by_column])

    values = collections.defaultdict(list) 
    values_from_get_dummies = collections.defaultdict(list)
    for key, item in grouped_df:
        # print (key)
        # print(grouped_df.get_group(key), "\n\n")
        item_data = grouped_df.get_group(key)
        # print (item_data.columns)
        # print (item_data['weight'].mean())
        # break
        values[group_by_column].append(key)
        values['weight'].append(item_data['weight'].mean()) 
        values['rating'].append(item_data['rating'].mean()) 
        values['height'].append(item_data['height'].mean()) 
        values['age'].append(item_data['age'].mean()) 
        total_length = len(item_data) 
        for col in item_data.columns:
            if col in values:
                continue 
            total = sum(item_data[col])
            if total > 1:
                values_from_get_dummies[col].append(1)
            else:
                values_from_get_dummies[col].append(0)
      # break

    profile_data = pd.DataFrame()
    profile_data[group_by_column] = values[group_by_column]
    profile_data['weight'] = values['weight']
    profile_data['rating'] = values['rating']
    profile_data['height'] = values['height']
    profile_data['age'] = values['age']
    for col in values_from_get_dummies:
        profile_data[col] = values_from_get_dummies[col]

    return profile_data

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
X = clothing_data.copy()
X_object = X.select_dtypes('object')
ohe.fit(X_object)
codes = ohe.transform(X_object).toarray()

In [ ]:
feature_names = ohe.get_feature_names(['fit', 'rented for', 'body type', 'category', 'size', 'bust_size', 'bust_type'])

clothing_data_for_item_profile = pd.concat([X.select_dtypes(exclude='object'), 
                                            pd.DataFrame(codes,columns=feature_names).astype(int)], axis=1)
clothing_data_for_item_profile
item_profile = get_profile('item_id', clothing_data_for_item_profile)
item_profile

In [ ]:
def get_user_data():
#     ['fit', 'weight', 'rented for', 'body type', 'category', 'height', 'size', 'age', 'bust_size', 'bust_type']
    input_values = {}
    
    age = 0
    while age <= 0 or age > 100 :
        age = input("Enter age: (0 to 100)")
        try:
            age = int(age)
        except:
            age = 0
    input_values['age'] = age
    
    height = 0
    while height <= 0 or height > 96:
        height = input("Enter height: (in inches)")
        try:
            height = int(height)
        except:
            height = 0
    input_values['height'] = height
    
    weight = 0
    while weight <= 0 or weight > 500:
        weight = input("Enter weight: (in lbs)")
        try:
            weight = int(weight)
        except:
            weight = 0
    input_values['weight'] = weight
    
    fit_values = ['fit', 'small', 'large']
    user_fit = None
    while user_fit not in fit_values:
        user_fit = input("Select fit: (" + ",".join(fit_values) + ")")
    input_values['fit'] = user_fit
    
    rented_for_values = ['vacation', 'other', 'party', 'formal affair', 'wedding', 'date', 'everyday', 'work', 'party: cocktail']
    rented_for = None
    while rented_for not in rented_for_values:
        rented_for = input("Select rented for: (" + ",".join(rented_for_values) + ")")
    input_values['rented_for'] = rented_for
    
    body_type_values = ['hourglass', 'straight & narrow', 'petite', 'pear', 'athletic', 'full bust', 'apple']
    body_type = None
    while body_type not in body_type_values:
        body_type = input("Select body type: (" + ",".join(body_type_values) + ")")
    input_values['body_type'] = body_type
    
    category_values = ['romper', 'gown', 'shift', 'dress', 'leggings', 'top', 'jumpsuit', 'sweater', 'jacket', 'shirtdress', 'maxi', 'pants', 'shirt', 'mini', 'skirts', 'Others', 'blouse', 'suit', 'coat', 'bomber', 'blazer', 'vest', 'tank', 'frock', 'tunic', 'cardigan', 'culottes', 'print', 'sweatshirt']
    category = None
    while category not in category_values:
        category = input("Select category: (" + ",".join(category_values) + ")")
    input_values['category'] = category
    
    size_values = ['M', 'XS', 'S', 'XL', 'L']
    size = None
    while size not in size_values:
        size = input("Select size: (" + ",".join(size_values) + ")")
    input_values['size'] = size
    
    bust_size = 0
    while bust_size < 28 or bust_size > 48 or bust_size % 2 != 0 :
        bust_size = input("Select bust size: (28 to 48)")
        try:
            bust_size = int(bust_size)
        except:
            bust_size = 0
    input_values['bust_size'] = str(float(bust_size))
    
    bust_type_values = ['d', 'b', 'c', 'a', 'dd', 'ddd', 'f', 'g', 'aa', 'i', 'h', 'j']
    bust_type = None
    while bust_type not in bust_type_values:
        bust_type = input("Select bust type: (" + ",".join(bust_type_values) + ")")
    input_values['bust_type'] = bust_type
    
    return input_values
    
user_input = get_user_data()
user_input

In [1]:
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity

def find_jaccard_similarity(item_profile, user_profile):
    item_id = []
    score = []
    for index, row in item_profile.iterrows():
        item_id.append(row['item_id'])
        del row['item_id']
        item_values = list(row.values)
        score.append(jaccard_score(item_values, user_profile, average='micro'))
        # break

    jaccard_similarity_scores = pd.DataFrame()
    jaccard_similarity_scores['item_id'] = item_id
    jaccard_similarity_scores['jaccard_score'] = score
    return jaccard_similarity_scores



def find_cosine_similarity(item_profile, user_profile):
    item_id = []
    score = []
    for index, row in item_profile.iterrows():
        item_id.append(row['item_id'])
        del row['item_id']
        score.append(cosine_similarity([row], [user_profile])[0][0])

    cosine_similarity_scores = pd.DataFrame()
    cosine_similarity_scores['item_id'] = item_id
    cosine_similarity_scores['cosine_similarity'] = score
    return cosine_similarity_scores


In [ ]:
# Cosine and jaccard 

continuous_values = item_profile[['item_id', 'weight', 'height', 'age']].copy()
discrete_values = item_profile.drop(['weight', 'rating', 'height', 'age'], axis = 1)

# ['fit', 'rented for', 'body type', 'category', 'size', 'bust_size', 'bust_type']
text_cols = ['fit', 'rented_for', 'body_type', 'category', 'size', 'bust_size', 'bust_type']
text_values = []
for col in text_cols:
    text_values.append(user_input[col])
text_data = list(ohe.transform([text_values]).toarray()[0])
jaccard_similarity_scores = find_jaccard_similarity(discrete_values, text_data)
jaccard_similarity_scores


# ['weight', 'height', 'age']
numeric_cols = ['weight', 'height', 'age']
numeric_data = []
for col in numeric_cols:
    numeric_data.append(user_input[col])
cosine_similarity_scores = find_cosine_similarity(continuous_values, numeric_data)
cosine_similarity_scores
all_scores = jaccard_similarity_scores.join(cosine_similarity_scores.set_index('item_id'), on='item_id')
all_scores['average'] = all_scores.apply(lambda x : (x['jaccard_score'] + x['cosine_similarity']) / 2, axis = 1)
item_ratings = item_profile[['item_id', 'rating']].copy()
all_scores = all_scores.join(item_ratings.set_index('item_id'), on='item_id')
all_scores = all_scores.sort_values(by=['cosine_similarity', 'rating'], ascending = False)

all_scores

In [ ]:
# ['fit', 'rented for', 'body type', 'category', 'size', 'bust_size', 'bust_type']
text_cols = ['fit', 'rented_for', 'body_type', 'category', 'size', 'bust_size', 'bust_type']
text_data = []
for col in text_cols:
    text_data.append(user_input[col])
text_data = list(ohe.transform([text_data]).toarray()[0])

# ['weight', 'height', 'age']
numeric_cols = ['weight', 'height', 'age']
numeric_data = []
for col in numeric_cols:
    numeric_data.append(user_input[col])
    
user_data = numeric_data + text_data

item_data = item_profile.drop(['rating'], axis = 1)
cosine_similarity_scores = find_cosine_similarity(item_data, user_data)
cosine_similarity_scores
item_ratings = item_profile[['item_id', 'rating']].copy()
all_scores = cosine_similarity_scores.join(item_ratings.set_index('item_id'), on='item_id')
all_scores = all_scores.sort_values(by=['cosine_similarity', 'rating'], ascending = False)

all_scores

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

location = "./train_stratify.csv"
clothing_data = pd.read_csv(location)
clothing_data

,fit,user_id,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,bust_size,bust_type
0,small,379417,658706,120.000000,5.0,party,"It ran a little small, in my opinion, but the ...",straight & narrow,"Perfect, Unique Dress!",dress,65.0,M,34.0,34.0,b
1,fit,904971,182915,120.000000,5.0,party,Wore this to a Christmas party - so comfortabl...,straight & narrow,Great fit and material!,dress,64.0,S,52.0,34.0,b
2,small,163215,1979533,150.000000,5.0,other,As the temperatures climbed into the 90s yeste...,hourglass,Bright and fun for engagement pics!,dress,69.0,S,28.0,34.0,d
3,small,232640,880935,140.000000,4.0,wedding,The dress color was a wow! I found the dress ...,hourglass,This dress is a standout!,dress,68.0,M,38.0,34.0,c
4,fit,691117,180014,137.391709,5.0,party,"This dress is very flattering, and the cutouts...",athletic,So easy to wear and comfy! Paired with red acc...,dress,69.0,L,33.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154030,small,733207,2358935,128.000000,4.0,work,This was a cute top. I really liked the lace a...,athletic,Fun Top.,top,66.0,S,38.0,32.0,dd
154031,fit,873762,625911,160.000000,5.0,wedding,This dress was great! The style of the dress i...,athletic,"Classic design, with an edge.",dress,68.0,XL,27.0,36.0,b
154032,fit,14227,1515649,145.000000,4.0,party,no complaints,pear,fit like a glove,dress,65.0,M,37.0,32.0,d
154033,fit,421813,127081,170.000000,4.0,wedding,The fit of the dress is pretty true to size. I...,hourglass,Cancelled wedding. but gorgeous dress,gown,66.0,XL,34.0,36.0,d


In [3]:
clothing_data = clothing_data.drop(['user_id', 'review_text', 'review_summary'], axis=1)
clothing_data = clothing_data.astype({'bust_size': 'str'})
clothing_data

,fit,item_id,weight,rating,rented for,body type,category,height,size,age,bust_size,bust_type
0,small,658706,120.000000,5.0,party,straight & narrow,dress,65.0,M,34.0,34.0,b
1,fit,182915,120.000000,5.0,party,straight & narrow,dress,64.0,S,52.0,34.0,b
2,small,1979533,150.000000,5.0,other,hourglass,dress,69.0,S,28.0,34.0,d
3,small,880935,140.000000,4.0,wedding,hourglass,dress,68.0,M,38.0,34.0,c
4,fit,180014,137.391709,5.0,party,athletic,dress,69.0,L,33.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...
154030,small,2358935,128.000000,4.0,work,athletic,top,66.0,S,38.0,32.0,dd
154031,fit,625911,160.000000,5.0,wedding,athletic,dress,68.0,XL,27.0,36.0,b
154032,fit,1515649,145.000000,4.0,party,pear,dress,65.0,M,37.0,32.0,d
154033,fit,127081,170.000000,4.0,wedding,hourglass,gown,66.0,XL,34.0,36.0,d


In [4]:
clothing_data_new = clothing_data.copy()
clothing_data_new['weight'] = clothing_data_new['weight'] // 10
clothing_data_new['age'] = clothing_data_new['age'] // 10 
clothing_data_new = clothing_data_new.astype({'weight': 'str', 'age': 'str', 'height': 'str'})
clothing_data_new

,fit,item_id,weight,rating,rented for,body type,category,height,size,age,bust_size,bust_type
0,small,658706,12.0,5.0,party,straight & narrow,dress,65.0,M,3.0,34.0,b
1,fit,182915,12.0,5.0,party,straight & narrow,dress,64.0,S,5.0,34.0,b
2,small,1979533,15.0,5.0,other,hourglass,dress,69.0,S,2.0,34.0,d
3,small,880935,14.0,4.0,wedding,hourglass,dress,68.0,M,3.0,34.0,c
4,fit,180014,13.0,5.0,party,athletic,dress,69.0,L,3.0,34.0,b
...,...,...,...,...,...,...,...,...,...,...,...,...
154030,small,2358935,12.0,4.0,work,athletic,top,66.0,S,3.0,32.0,dd
154031,fit,625911,16.0,5.0,wedding,athletic,dress,68.0,XL,2.0,36.0,b
154032,fit,1515649,14.0,4.0,party,pear,dress,65.0,M,3.0,32.0,d
154033,fit,127081,17.0,4.0,wedding,hourglass,gown,66.0,XL,3.0,36.0,d


In [5]:
from sklearn.preprocessing import OneHotEncoder
ohe_new = OneHotEncoder(handle_unknown='ignore')
X_new = clothing_data_new.copy()
X_object_new = X_new.select_dtypes('object')
ohe_new.fit(X_object_new)
codes_new = ohe_new.transform(X_object_new).toarray()
feature_names_new = ohe_new.get_feature_names(['fit', 'weight', 'rented for', 'body type', 'category', 'height', 'size', 'age', 'bust_size', 'bust_type'])
items_new = pd.concat([X_new.select_dtypes(exclude='object'), 
                        pd.DataFrame(codes_new, columns = feature_names_new).astype(int)], axis=1)

items_new

,item_id,rating,fit_fit,fit_large,fit_small,weight_10.0,weight_11.0,weight_12.0,weight_13.0,weight_14.0,...,bust_type_b,bust_type_c,bust_type_d,bust_type_dd,bust_type_ddd,bust_type_f,bust_type_g,bust_type_h,bust_type_i,bust_type_j
0,658706,5.0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,182915,5.0,1,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,1979533,5.0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,880935,4.0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
4,180014,5.0,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154030,2358935,4.0,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
154031,625911,5.0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
154032,1515649,4.0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
154033,127081,4.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [6]:
import collections

def get_item_profile(df):
    values_from_get_dummies = collections.defaultdict(list)
    item_id = []
    ratings = []
    grouped_items = df.groupby(['item_id'])
    for key, item in grouped_items:

        item_data = grouped_items.get_group(key)
        item_id.append(key)
        ratings.append(item['rating'].mean())
        for col in item_data.columns:
            if col == 'item_id' or col == 'rating':
                continue
            total = sum(item_data[col])
            if total > 1:
                values_from_get_dummies[col].append(1)
            else:
                values_from_get_dummies[col].append(0)


    profile_data = pd.DataFrame()
    profile_data['item_id'] = item_id
    profile_data['rating'] = ratings
    for col in values_from_get_dummies:
        profile_data[col] = values_from_get_dummies[col]
    return profile_data

In [7]:
items_profile_new = get_item_profile(items_new)
items_profile_new

,item_id,rating,fit_fit,fit_large,fit_small,weight_10.0,weight_11.0,weight_12.0,weight_13.0,weight_14.0,...,bust_type_b,bust_type_c,bust_type_d,bust_type_dd,bust_type_ddd,bust_type_f,bust_type_g,bust_type_h,bust_type_i,bust_type_j
0,123373,4.404719,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
1,123793,4.766445,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
2,124204,4.621939,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,0
3,124553,4.278515,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
4,125424,4.501333,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5757,2963850,4.875000,1,0,0,0,0,0,1,0,...,1,1,0,1,0,0,0,0,0,0
5758,2964470,5.000000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5759,2965009,4.666667,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5760,2965924,4.666667,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [8]:
item_profile = items_profile_new.drop(['rating'], axis = 1)
item_profile

,item_id,fit_fit,fit_large,fit_small,weight_10.0,weight_11.0,weight_12.0,weight_13.0,weight_14.0,weight_15.0,...,bust_type_b,bust_type_c,bust_type_d,bust_type_dd,bust_type_ddd,bust_type_f,bust_type_g,bust_type_h,bust_type_i,bust_type_j
0,123373,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
1,123793,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
2,124204,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,0
3,124553,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
4,125424,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5757,2963850,1,0,0,0,0,0,1,0,0,...,1,1,0,1,0,0,0,0,0,0
5758,2964470,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5759,2965009,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5760,2965924,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
import evaluation as evalResult
from sklearn.metrics import jaccard_score

def findRecommendataions(user_input):
    # ['fit', 'weight', 'rented for', 'body type', 'category', 'height', 'size', 'age', 'bust_size', 'bust_type']
    all_cols = ['fit', 'weight', 'rented for', 'body type', 'category', 'height', 'size', 'age', 'bust_size', 'bust_type']
    user_data = []
    for col in all_cols:
        user_data.append(user_input[col])
    user_data = list(ohe_new.transform([user_data]).toarray()[0])
    test_score = find_jaccard_similarity(item_profile, user_data)
    # test_score
    item_ratings = items_profile_new[['item_id', 'rating']].copy()
    test_score = test_score.join(item_ratings.set_index('item_id'), on='item_id')
    test_score = test_score.sort_values(by=['jaccard_score', 'rating'], ascending = False)
    return list(test_score['item_id'])

evalResult.evaluate_recommendation(findRecommendataions)
# evalResult.test_method()

Test 0 : Item ID 208647 was recommended was your 5169 recommendation
Test 1 : Item ID 145906 was recommended was your 5746 recommendation
Test 2 : Item ID 1740815 was recommended was your 2088 recommendation
Test 3 : Item ID 2803807 was recommended was your 1391 recommendation
Test 4 : Item ID 2521411 was recommended was your 530 recommendation
Test 5 : Item ID 1198628 was recommended was your 4087 recommendation
Test 6 : Item ID 492247 was recommended was your 4272 recommendation
Test 7 : Item ID 1832150 was recommended was your 4211 recommendation
Test 8 : Item ID 2224766 was recommended was your 1159 recommendation
Test 9 : Item ID 319191 was recommended was your 4394 recommendation
Test 10 : Item ID 2155094 was recommended was your 5381 recommendation
Test 11 : Item ID 963476 was recommended was your 5613 recommendation
Test 12 : Item ID 1435687 was recommended was your 4252 recommendation
Test 13 : Item ID 1810862 was recommended was your 999 recommendation
Test 14 : Item ID 15939

Test 117 : Item ID 1814462 was recommended was your 4891 recommendation
Test 118 : Item ID 883661 was recommended was your 5720 recommendation
Test 119 : Item ID 1989948 was recommended was your 4513 recommendation
Test 120 : Item ID 1147534 was recommended was your 1066 recommendation
Test 121 : Item ID 862815 was recommended was your 4070 recommendation
Test 122 : Item ID 280443 was recommended was your 1983 recommendation
Test 123 : Item ID 832622 was recommended was your 5232 recommendation
Test 124 : Item ID 2846069 was recommended was your 4182 recommendation
Test 125 : Item ID 2843850 was recommended was your 318 recommendation
Test 126 : Item ID 450618 was recommended was your 5668 recommendation
Test 127 : Item ID 1460767 was recommended was your 4366 recommendation
Test 128 : Item ID 1940267 was recommended was your 3634 recommendation
Test 129 : Item ID 936038 was recommended was your 2814 recommendation
Test 130 : Item ID 1581933 was recommended was your 4211 recommendation

Test 232 : Item ID 561864 was recommended was your 4602 recommendation
Test 233 : Item ID 154002 was recommended was your 5724 recommendation
Test 234 : Item ID 1844844 was recommended was your 3967 recommendation
Test 235 : Item ID 1429022 was recommended was your 5415 recommendation
Test 236 : Item ID 1366101 was recommended was your 5342 recommendation
Test 237 : Item ID 1695279 was recommended was your 5705 recommendation
Test 238 : Item ID 130259 was recommended was your 5758 recommendation
Test 239 : Item ID 126335 was recommended was your 5762 recommendation
Test 240 : Item ID 1800907 was recommended was your 4655 recommendation
Test 241 : Item ID 1692683 was recommended was your 5235 recommendation
Test 242 : Item ID 534612 was recommended was your 4671 recommendation
Test 243 : Item ID 2821045 was recommended was your 4282 recommendation
Test 244 : Item ID 226072 was recommended was your 4593 recommendation
Test 245 : Item ID 288472 was recommended was your 5299 recommendation

Test 347 : Item ID 2029039 was recommended was your 2879 recommendation
Test 348 : Item ID 561864 was recommended was your 4420 recommendation
Test 349 : Item ID 1692935 was recommended was your 5356 recommendation
Test 350 : Item ID 1283795 was recommended was your 3447 recommendation
Test 351 : Item ID 1408079 was recommended was your 5049 recommendation
Test 352 : Item ID 1982555 was recommended was your 4110 recommendation
Test 353 : Item ID 2932257 was recommended was your 4548 recommendation
Test 354 : Item ID 2456857 was recommended was your 3523 recommendation
Test 355 : Item ID 159931 was recommended was your 5413 recommendation
Test 356 : Item ID 1463543 was recommended was your 4608 recommendation
Test 357 : Item ID 851603 was recommended was your 4774 recommendation
Test 358 : Item ID 1769705 was recommended was your 5460 recommendation
Test 359 : Item ID 1449570 was recommended was your 5440 recommendation
Test 360 : Item ID 123793 was recommended was your 5757 recommendat

Test 462 : Item ID 631251 was recommended was your 4475 recommendation
Test 463 : Item ID 2281848 was recommended was your 4159 recommendation
Test 464 : Item ID 136110 was recommended was your 5751 recommendation
Test 465 : Item ID 1697200 was recommended was your 5623 recommendation
Test 466 : Item ID 190529 was recommended was your 4302 recommendation
Test 467 : Item ID 2522622 was recommended was your 3499 recommendation
Test 468 : Item ID 1687082 was recommended was your 5615 recommendation
Test 469 : Item ID 152998 was recommended was your 5353 recommendation
Test 470 : Item ID 132738 was recommended was your 5761 recommendation
Test 471 : Item ID 1835732 was recommended was your 4358 recommendation
Test 472 : Item ID 1786364 was recommended was your 3934 recommendation
Test 473 : Item ID 1260666 was recommended was your 5112 recommendation
Test 474 : Item ID 162634 was recommended was your 5589 recommendation
Test 475 : Item ID 1854157 was recommended was your 3784 recommendatio

Test 577 : Item ID 921642 was recommended was your 5687 recommendation
Test 578 : Item ID 1005880 was recommended was your 5247 recommendation
Test 579 : Item ID 1973037 was recommended was your 5016 recommendation
Test 580 : Item ID 925104 was recommended was your 4374 recommendation
Test 581 : Item ID 2708578 was recommended was your 3191 recommendation
Test 582 : Item ID 174086 was recommended was your 5720 recommendation
Test 583 : Item ID 655775 was recommended was your 5167 recommendation
Test 584 : Item ID 125465 was recommended was your 5759 recommendation
Test 585 : Item ID 2796868 was recommended was your 3660 recommendation
Test 586 : Item ID 1290861 was recommended was your 2567 recommendation
Test 587 : Item ID 127865 was recommended was your 5751 recommendation
Test 588 : Item ID 1708327 was recommended was your 1713 recommendation
Test 589 : Item ID 1476825 was recommended was your 5060 recommendation
Test 590 : Item ID 337074 was recommended was your 3037 recommendation